# Source: NCAA

In [1]:
#hide
import core_constants as cc
import functions as fx
import json
import pandas as pd
import sqlite3 as sql

## Set Notebook Settings

In [ ]:
#Load NCAA Site Schema
schoolsList = json.loads(open('..//config//schools.json', "r").read())
ncaaDates = json.loads(open('..//config//ncaa_dates.json', "r").read())

## Get & Save the Teams Page HTML
#### Source: https://stats.ncaa.org/team/392/roster/15280
> This page contains a roster for any given team in any year (using their own ids)

In [ ]:
fx.get_NCAA(schoolsList, ncaaDates)

## Process Local NCAA HTML Files

> All of this processing is done locally, using the files saved in the previous few steps.  This creates an exhaustive store of all the fields grabbed from the scrapes.

In [ ]:
conferences = cc.get_availableConferences()

cc.save_records('scrapedData', 'ncaa', fx.process_NCAA(conferences))

## Save to DB

In [3]:
fx.toDB_NCAA()

'DB Write is done'